## Imports

In [14]:
# General imports
import json
import random
import pandas as pd
from transformers import set_seed


# Intent recognition imports
import torch
torch.manual_seed(42)

# ChatGPT imports
%load_ext autoreload
%autoreload 2
from gpt import GPT
from settings import Settings
from intent_model import IntentModel
from chatbot import ChatbotLogic

# Chat demo
import gradio as gr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
random.seed(SEED)
set_seed(SEED)

# Load data 

In [3]:
# Load the data from JSON file
with open("training_data/intents/intent_recognition.json", "r") as f:
    data = json.load(f)

validation_data = []
train_data = []
for datum in data:
    texts = datum["train_questions"]
    label = datum["intent"]
    validation = datum["test_questions"]
    for text in texts:
        train_data.append((text, label))
    for text in validation:
        validation_data.append((text, label))

# Concatenate train and validation data for generating label_map
all_data = train_data + validation_data

# Define the mapping between top-level labels and integers
# Sort the labels before enumerating
label_map = {label: i for i, label in enumerate(sorted(set([data[1] for data in all_data])))}

# Convert the training data labels to integers using the label_map
train_labels = torch.tensor([label_map[data[1]] for data in train_data])
validation_labels = torch.tensor([label_map[data[1]] for data in validation_data])

# Intent recognition model

In [4]:
intent_model = IntentModel(train_data, validation_data, label_map, train_labels, validation_labels)

In [5]:
print(intent_model.get_intent("Can you provide details on the rules pertaining to the age limit for using the company's services?")[0], ": minimum age")
print(intent_model.get_intent("Could you summarize the main provisions of the legal statement?")[0], ": legal_statement_information")
print(intent_model.get_intent("Are you allowed to transfer ownership of my account or subscription to another individual?")[0], ": transfer_legal_statement")

minimum_age : minimum age
legal_statement_information : legal_statement_information
transfer_legal_statement : transfer_legal_statement


In [6]:
# test the intent model
predicted_intent, confidence_scores = intent_model.get_intent("Can you provide details on the rules pertaining to the age limit for using the company's services?")

# Retrieve the intent label and its corresponding confidence score
highest_confidence_intent = predicted_intent

print("Highest confidence intent:", highest_confidence_intent)
print("Highest confidence score:", confidence_scores)
print(len(confidence_scores))
print(len(intent_model.label_map.keys()))

pd.DataFrame({
    'keys': intent_model.label_map.keys(),
    'confidences': confidence_scores
})

Highest confidence intent: minimum_age
Highest confidence score: [0.004718465264886618, 0.008686626330018044, 0.006868220865726471, 0.0030268209520727396, 0.003119484055787325, 0.004925729241222143, 0.0025988933630287647, 0.0021945328917354345, 0.0051748245023190975, 0.002587398746982217, 0.0013433032436296344, 0.9125367403030396, 0.0036402964033186436, 0.009336018934845924, 0.004967961926013231, 0.003311765845865011, 0.0033700952772051096, 0.0018986603245139122, 0.00253856903873384, 0.001898297225125134, 0.011257492005825043]
21
21


,keys,confidences
0,access_personal_data,0.004718
1,cancellation_legal_statement,0.008687
2,contact_privacy_concerns,0.006868
3,cookie_policy,0.003027
4,data_breaches,0.003119
5,data_collection,0.004926
6,data_sharing_opt_out,0.002599
7,data_storage,0.002195
8,delete_personal_information,0.005175
9,hello_response,0.002587


## ChatGPT model

In [7]:
# GPT model here
gpt_model = GPT()

privacy_level = 1 # set to 2 for maximum privacy
minimum_confidence_score = 0.4

# Load Settings class

In [11]:
settings_model = Settings()

# Test the settings model
# settings_model.update_user_setting("1", "profile_published", False)
settings_model.set_user_calling_card_visibility(user_id = "0", individual_user_id = "2", profile_card_component = "profile_card_component_3_visible", component_state = True)

# Set up gradio chat for demo

In [23]:
theme = gr.themes.Soft(
    primary_hue="orange",
    secondary_hue="orange",
).set(
    body_text_color_dark='*neutral_800',
    background_fill_primary_dark='*neutral_50',
    background_fill_secondary_dark='*neutral_50',
    border_color_accent_dark='*primary_300',
    border_color_primary_dark='*neutral_200',
    color_accent_soft_dark='*primary_50',
    link_text_color_dark='*secondary_600',
    link_text_color_active_dark='*secondary_600',
    link_text_color_hover_dark='*secondary_700',
    link_text_color_visited_dark='*secondary_500',
    block_background_fill='*neutral_100',
    block_background_fill_dark='*neutral_100',
    block_label_background_fill='*primary_400',
    block_label_background_fill_dark='*primary_400',
    block_label_text_color='*neutral_50',
    block_label_text_color_dark='*neutral_50',
    block_title_text_color='*neutral_50',
    block_title_text_color_dark='*neutral_50',
    checkbox_background_color_dark='*background_fill_primary',
    checkbox_background_color_selected='*primary_500',
    checkbox_background_color_selected_dark='*primary_500',
    checkbox_border_color_dark='*neutral_100',
    checkbox_border_color_focus='*primary_300',
    checkbox_border_color_focus_dark='*primary_300',
    checkbox_border_color_hover_dark='*neutral_300',
    checkbox_border_color_selected='*primary_500',
    checkbox_border_color_selected_dark='*primary_500',
    checkbox_border_width_dark='1px',
    checkbox_label_background_fill_selected_dark='*primary_500',
    checkbox_label_text_color_selected_dark='white',
    error_background_fill_dark='#fee2e2',
    error_border_color_dark='#fecaca',
    input_background_fill_dark='white',
    input_background_fill_focus_dark='*secondary_500',
    input_border_color_dark='*neutral_50',
    input_border_color_focus_dark='*secondary_300',
    input_placeholder_color_dark='*neutral_400',
    slider_color_dark='*primary_500',
    stat_background_fill_dark='*primary_300',
    table_border_color_dark='*neutral_300',
    table_even_background_fill_dark='white',
    table_odd_background_fill_dark='*neutral_50',
    button_primary_background_fill_dark='*primary_500',
    button_primary_background_fill_hover_dark='*primary_400',
    button_primary_border_color_dark='*primary_200',
    button_secondary_background_fill_dark='white',
    button_secondary_background_fill_hover_dark='*neutral_100',
    button_secondary_border_color_dark='*neutral_200',
    button_secondary_text_color_dark='*neutral_800'
)

def initialize_chat():
    return ChatbotLogic(data, gpt_model, intent_model, minimum_confidence_score, privacy_level)

with gr.Blocks(theme=theme, css="assets/chat.css") as demo:
    chatbot_logic = initialize_chat()

    gr.Image("https://iyyu.com/_nuxt/img/navbar_logoW@2x.79eba99.png", interactive=False, tool="image", show_label=False, elem_classes="logo").style(width=200)
    chatbot = gr.Chatbot(chatbot_logic.history)
    msg = gr.Textbox()
    submit = gr.Button("Submit")

    msg.submit(chatbot_logic.user, [msg, chatbot], [msg, chatbot], queue=False).then(
        chatbot_logic.bot, chatbot, chatbot
    )
    submit.click(chatbot_logic.user, [msg, chatbot], [msg, chatbot], queue=False).then(
        chatbot_logic.bot, chatbot, chatbot
    )

demo.launch()

TypeError: ChatbotLogic.__init__() missing 5 required positional arguments: 'data', 'gpt_model', 'intent_model', 'minimum_confidence_score', and 'privacy_level'